# **Page 1 - CM's Performance**

In [1]:
import pyodbc
import pandas as pd
import psycopg2
import time 
from datetime import datetime
import calendar
import openpyxl

In [2]:
sql_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=SDPL-IC-PBGW-VM;'
    'DATABASE=SdplWarehouse;'
    'UID=lisadmin;'
    'PWD=abcd-1234'
)

In [3]:
df = pd.read_sql("SELECT * FROM VW_CMP", sql_conn)
df.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\3468086678.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM VW_CMP", sql_conn)


,SITEID,COSTCENTER,SUBZONENAME,FM,TARGET,ACTUAL_SALES
0,BLG,BALLYGUNGE,South Kol,May-25,2791959.0,3.116776e+06
1,HWH,HOWRAH,Howrah/Hooghly,May-25,1527092.0,1.553950e+06
2,TAM,TAMLUK,Midnapore,Jun-25,2046675.0,1.357441e+06
3,KHA,KHARDHA,Khardah Zone,May-25,12654132.0,1.372658e+07
4,BAN,BANGAON,Barasat Zone,May-25,3213045.0,3.539111e+06


In [4]:
dff = pd.read_sql("SELECT * FROM VW_CMP_DU", sql_conn)
dff.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\3665727932.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dff = pd.read_sql("SELECT * FROM VW_CMP_DU", sql_conn)


,INVENTSITEID,FM,SRConvperc
0,ASN,Apr-25,45.446341
1,BAN,Apr-25,26.339523
2,BAR,Apr-25,42.661717
3,BAS,Apr-25,16.478107
4,BEH,Apr-25,42.387863


In [5]:
cmdf = pd.merge(df, dff, left_on= ['SITEID', 'FM'], right_on= ['INVENTSITEID', 'FM'], how= 'left')

cmdf['ACTUAL_SALES'] = cmdf['ACTUAL_SALES']

cmdf = cmdf[['SITEID', 'COSTCENTER', 'SUBZONENAME', 'FM', 'TARGET', 'ACTUAL_SALES', 'SRConvperc']].sort_values(by= 'SITEID', ascending= True)
cmdf

,SITEID,COSTCENTER,SUBZONENAME,FM,TARGET,ACTUAL_SALES,SRConvperc
101,ASN,ASANSOL,Burdwan,Jun-25,4552577.0,3464518.04,53.971256
113,ASN,ASANSOL,Burdwan,May-25,4601248.0,3913940.77,31.510074
122,ASN,ASANSOL,Burdwan,Apr-25,4303131.0,3391092.60,45.446341
42,BAN,BANGAON,Barasat Zone,Jun-25,3179057.0,2912362.01,49.899945
4,BAN,BANGAON,Barasat Zone,May-25,3213045.0,3539110.72,56.768897
...,...,...,...,...,...,...,...
176,TLG,TOLLYGUNGE,South Kol,Jun-25,6141119.0,4993913.18,46.894701
130,TLG,TOLLYGUNGE,South Kol,Apr-25,5804634.0,5470502.47,22.909447
44,TML,TAMLUK LAB,Midnapore,None,NaN,NaN,NaN
31,ULB,Uluberia,Howrah/Hooghly,None,NaN,23840.00,NaN


In [8]:
filled = pd.read_sql("""SELECT FORMAT(SUBMITTIME, 'MMM-yy') AS FM, INVENTSITEID, 
COUNT(DISTINCT CASE WHEN RATINGVALUE != '' THEN SRNO END) AS TotalFeedbackFilled
FROM VW_REPORT_FEEDBACK
GROUP BY FORMAT(SUBMITTIME, 'MMM-yy'), INVENTSITEID""", sql_conn)
filled.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\640488886.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  filled = pd.read_sql("""SELECT FORMAT(SUBMITTIME, 'MMM-yy') AS FM, INVENTSITEID,


,FM,INVENTSITEID,TotalFeedbackFilled
0,Jan-25,DIH,327
1,Dec-24,ASN,88
2,Jun-25,BRP,12
3,Nov-24,SNT,104
4,May-25,HBT,73


In [9]:
totalid = pd.read_sql("""SELECT FORMAT(TRANSDATE, 'MMM-yy') AS FM, COSTCENTER, COUNT(DISTINCT(SALESID)) TOTAL_SALESID
FROM [VW_BI_SALES24-25]
WHERE TRANSDATE >= '2025-04-01'
GROUP BY FORMAT(TRANSDATE, 'MMM-yy'), COSTCENTER""", sql_conn)
totalid.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\986089695.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  totalid = pd.read_sql("""SELECT FORMAT(TRANSDATE, 'MMM-yy') AS FM, COSTCENTER, COUNT(DISTINCT(SALESID)) TOTAL_SALESID


,FM,COSTCENTER,TOTAL_SALESID
0,Jun-25,BRP,141
1,May-25,HBT,771
2,May-25,DUR,2374
3,Jun-25,KHA,5608
4,Apr-25,DIH,2302


In [10]:
xdf = pd.merge(cmdf, totalid, left_on = ['SITEID', 'FM'], right_on = ['COSTCENTER', 'FM'], how= 'left')
xdf = xdf[['SITEID', 'COSTCENTER_x', 'SUBZONENAME', 'FM', 'TARGET', 'ACTUAL_SALES', 'SRConvperc', 'TOTAL_SALESID']]
xdf.head()

,SITEID,COSTCENTER_x,SUBZONENAME,FM,TARGET,ACTUAL_SALES,SRConvperc,TOTAL_SALESID
0,ASN,ASANSOL,Burdwan,Jun-25,4552577.0,3464518.04,53.971256,2366.0
1,ASN,ASANSOL,Burdwan,May-25,4601248.0,3913940.77,31.510074,2815.0
2,ASN,ASANSOL,Burdwan,Apr-25,4303131.0,3391092.60,45.446341,2584.0
3,BAN,BANGAON,Barasat Zone,Jun-25,3179057.0,2912362.01,49.899945,1592.0
4,BAN,BANGAON,Barasat Zone,May-25,3213045.0,3539110.72,56.768897,1988.0


In [11]:
fdbk = pd.merge(xdf, filled, left_on= ['SITEID', 'FM'], right_on= ['INVENTSITEID', 'FM'], how= 'left')
fdbk['feedback_perticipation'] = fdbk['TotalFeedbackFilled'] * 100.0 / fdbk['TOTAL_SALESID']
fdbk= fdbk[['SITEID', 'COSTCENTER_x', 'SUBZONENAME', 'FM', 'TARGET', 'ACTUAL_SALES', 'SRConvperc', 'feedback_perticipation', 'TOTAL_SALESID']]
fdbk['SRConvperc'] = fdbk['SRConvperc'].round(2)
fdbk['feedback_perticipation'] = fdbk['feedback_perticipation'].round(2)
fdbk.head(7)

,SITEID,COSTCENTER_x,SUBZONENAME,FM,TARGET,ACTUAL_SALES,SRConvperc,feedback_perticipation,TOTAL_SALESID
0,ASN,ASANSOL,Burdwan,Jun-25,4552577.0,3464518.04,53.97,10.90,2366.0
1,ASN,ASANSOL,Burdwan,May-25,4601248.0,3913940.77,31.51,9.09,2815.0
2,ASN,ASANSOL,Burdwan,Apr-25,4303131.0,3391092.60,45.45,1.32,2584.0
3,BAN,BANGAON,Barasat Zone,Jun-25,3179057.0,2912362.01,49.90,21.92,1592.0
4,BAN,BANGAON,Barasat Zone,May-25,3213045.0,3539110.72,56.77,20.67,1988.0
5,BAN,BANGAON,Barasat Zone,Apr-25,3004870.0,3495875.67,26.34,1.69,1890.0
6,BAR,BARASAT,Barasat Zone,Apr-25,12448605.0,12608480.70,42.66,6.54,5303.0


In [12]:
#dynamic
today = datetime.today()
days_so_far = today.day
days_in_months = calendar.monthrange(today.year, today.month)[1]

In [13]:
fdbk['SalesProj'] = (fdbk['ACTUAL_SALES'] / 30) * 30
fdbk['TargetProj'] = fdbk['SalesProj'] * 100.0 / fdbk['TARGET']
fdbk['TargetProj'] = fdbk['TargetProj'].round(2)
fdbk = fdbk[['SITEID', 'COSTCENTER_x', 'SUBZONENAME', 'SalesProj', 'FM', 'TARGET', 'ACTUAL_SALES', 'TargetProj', 'SRConvperc', 'feedback_perticipation', 'TOTAL_SALESID']]
fdbk

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,feedback_perticipation,TOTAL_SALESID
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,4552577.0,3464518.04,76.10,53.97,10.90,2366.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,4601248.0,3913940.77,85.06,31.51,9.09,2815.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,4303131.0,3391092.60,78.81,45.45,1.32,2584.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,3179057.0,2912362.01,91.61,49.90,21.92,1592.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,3213045.0,3539110.72,110.15,56.77,20.67,1988.0
...,...,...,...,...,...,...,...,...,...,...,...
173,TLG,TOLLYGUNGE,South Kol,4993913.18,Jun-25,6141119.0,4993913.18,81.32,46.89,11.86,3407.0
174,TLG,TOLLYGUNGE,South Kol,5470502.47,Apr-25,5804634.0,5470502.47,94.24,22.91,1.12,3663.0
175,TML,TAMLUK LAB,Midnapore,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
176,ULB,Uluberia,Howrah/Hooghly,23840.00,None,NaN,23840.00,NaN,NaN,NaN,NaN


In [14]:
scores = pd.read_sql("""

SELECT 
    FM, INVENTSITEID, FeedbackFilled,
    CleScr * 10 AS CleFullScr, 
    CleRec, 
    AmbScr * 10 AS AmbFullScr, 
    AmbRec,
    ((CleScr * 10) + (AmbScr * 10)) AS TotalCAF, 
    (CleRec + AmbRec) AS TotalCAR,
    (CleRec + AmbRec) * 100.0 / NULLIF(((CleScr * 10) + (AmbScr * 10)), 0) AS CAScore, 
    Promoters, 
    Detractors, 
    NPSfilled,
    Promoters * 100.0 / NULLIF(NPSfilled, 0) AS PromPerc, 
    Detractors * 100.0 / NULLIF(NPSfilled, 0) AS DetrPerc,
    (Promoters * 100.0 / NULLIF(NPSfilled, 0)) - (Detractors * 100.0 / NULLIF(NPSfilled, 0)) AS NPSScore
FROM (
    SELECT 
        FORMAT(SUBMITTIME, 'MMM-yy') AS FM, 
        INVENTSITEID,

		COUNT(DISTINCT CASE WHEN RATINGVALUE != '' THEN SRNO END) AS FeedbackFilled,

        COUNT(CASE 
            WHEN QUESTIONNAME = 'How would you rate the overall cleanliness of the clinic?' 
                 AND TRY_CAST(RATINGVALUE AS int) IS NOT NULL 
            THEN QUESTIONNAME 
        END) AS CleScr,

        SUM(CASE 
            WHEN QUESTIONNAME = 'How would you rate the overall cleanliness of the clinic?' 
                 AND TRY_CAST(RATINGVALUE AS int) IS NOT NULL 
            THEN TRY_CAST(RATINGVALUE AS int) 
        END) AS CleRec,

        COUNT(CASE 
            WHEN QUESTIONNAME = 'How would you rate the ambiance of the clinic?' 
                 AND TRY_CAST(RATINGVALUE AS int) IS NOT NULL 
            THEN QUESTIONNAME 
        END) AS AmbScr,

        SUM(CASE 
            WHEN QUESTIONNAME = 'How would you rate the ambiance of the clinic?' 
                 AND TRY_CAST(RATINGVALUE AS int) IS NOT NULL 
            THEN TRY_CAST(RATINGVALUE AS int) 
        END) AS AmbRec,

        COUNT(CASE 
            WHEN QUESTIONNAME = 'How likely are you to recommend our clinic to a friend or family member?' 
                 AND TRY_CAST(RATINGVALUE AS int) IN (9,10) 
            THEN QUESTIONNAME 
        END) AS Promoters,

        COUNT(CASE 
            WHEN QUESTIONNAME = 'How likely are you to recommend our clinic to a friend or family member?' 
                 AND TRY_CAST(RATINGVALUE AS int) NOT IN (7,8,9,10) 
            THEN QUESTIONNAME 
        END) AS Detractors,

        COUNT(CASE 
            WHEN QUESTIONNAME = 'How likely are you to recommend our clinic to a friend or family member?' 
                 AND TRY_CAST(RATINGVALUE AS int) IS NOT NULL
            THEN QUESTIONNAME 
        END) AS NPSfilled

    FROM VW_REPORT_FEEDBACK
    GROUP BY FORMAT(SUBMITTIME, 'MMM-yy'), INVENTSITEID
) AS X

""", sql_conn)

scores.head(3)

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\3099925564.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scores = pd.read_sql("""


,FM,INVENTSITEID,FeedbackFilled,CleFullScr,CleRec,AmbFullScr,AmbRec,TotalCAF,TotalCAR,CAScore,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,NPSScore
0,Apr-25,None,7,60,53.0,70,54.0,130,107.0,82.307692,3,1,6,50.00000,16.666667,33.333333
1,Aug-24,None,1,0,NaN,0,NaN,0,NaN,NaN,0,0,0,NaN,NaN,NaN
2,Dec-24,None,412,4120,4053.0,4100,4026.0,8220,8079.0,98.284672,401,1,409,98.04401,0.244499,97.799511


In [15]:
fdf = pd.merge(fdbk, scores, left_on= ['SITEID', 'FM'], right_on= ['INVENTSITEID', 'FM'], how= 'left')
fdf = fdf[['SITEID', 'COSTCENTER_x', 'SUBZONENAME', 'SalesProj', 'FM', 'FeedbackFilled','TARGET', 'ACTUAL_SALES', 'TargetProj', 'SRConvperc', 'feedback_perticipation',
           'CAScore', 'NPSScore', 'CleFullScr', 'CleRec', 'AmbFullScr', 'AmbRec', 'Promoters', 'Detractors', 'NPSfilled', 'PromPerc',
           'DetrPerc', 'TOTAL_SALESID']]
fdf['PromPerc'] = fdf['PromPerc'].round(2)
fdf['DetrPerc'] = fdf['DetrPerc'].round(2)
fdf['CAScore'] = fdf['CAScore'].round(2)
fdf['NPSScore'] = fdf['NPSScore'].round(2)
fdf.head(10)

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,FeedbackFilled,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,...,CleFullScr,CleRec,AmbFullScr,AmbRec,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,TOTAL_SALESID
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,258.0,4552577.0,3464518.04,76.10,53.97,...,2470.0,2326.0,2290.0,2183.0,214.0,9.0,237.0,90.30,3.80,2366.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,256.0,4601248.0,3913940.77,85.06,31.51,...,2390.0,2150.0,2200.0,2007.0,201.0,15.0,226.0,88.94,6.64,2815.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,34.0,4303131.0,3391092.60,78.81,45.45,...,280.0,229.0,220.0,195.0,21.0,5.0,28.0,75.00,17.86,2584.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,349.0,3179057.0,2912362.01,91.61,49.90,...,3420.0,3358.0,3360.0,3324.0,328.0,9.0,340.0,96.47,2.65,1592.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,411.0,3213045.0,3539110.72,110.15,56.77,...,4010.0,3940.0,3950.0,3901.0,393.0,7.0,403.0,97.52,1.74,1988.0
5,BAN,BANGAON,Barasat Zone,3495875.67,Apr-25,32.0,3004870.0,3495875.67,116.34,26.34,...,280.0,255.0,230.0,190.0,16.0,2.0,20.0,80.00,10.00,1890.0
6,BAR,BARASAT,Barasat Zone,12608480.70,Apr-25,347.0,12448605.0,12608480.70,101.28,42.66,...,3280.0,3171.0,3200.0,3083.0,295.0,11.0,317.0,93.06,3.47,5303.0
7,BAR,BARASAT,Barasat Zone,10888172.57,Jun-25,814.0,13170230.0,10888172.57,82.67,64.97,...,7900.0,7588.0,7590.0,7336.0,718.0,29.0,772.0,93.01,3.76,4706.0
8,BAR,BARASAT,Barasat Zone,13293663.51,May-25,1022.0,13311033.0,13293663.51,99.87,54.94,...,9970.0,9467.0,9640.0,9240.0,896.0,42.0,975.0,91.90,4.31,5887.0
9,BAS,BASHIRHAT,Barasat Zone,4153041.53,May-25,348.0,4416430.0,4153041.53,94.04,26.56,...,3400.0,3306.0,3300.0,3256.0,324.0,8.0,336.0,96.43,2.38,2130.0


In [16]:
rls = pd.read_sql("""select SITEID, SITEEMAIL
from INVENTSITE
where SITEEMAIL != ''
""", sql_conn)
rls.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_12552\465647572.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rls = pd.read_sql("""select SITEID, SITEEMAIL


,SITEID,SITEEMAIL
0,ASN,asansolmgr@surakshanet.com
1,BAN,bangaonmgr@surakshanet.com
2,BAR,barasatmgr@surakshanet.com
3,BAS,basirhatmgr@surakshanet.com
4,BEH,behalamgr@surakshanet.com


In [17]:
fdf = pd.merge(fdf, rls, on= 'SITEID', how= 'left')

fdf = fdf[['SITEID', 'COSTCENTER_x', 'SUBZONENAME', 'SalesProj', 'FM',  'FeedbackFilled','TARGET', 'ACTUAL_SALES', 'TargetProj', 'SRConvperc', 'feedback_perticipation',
           'CAScore', 'NPSScore', 'CleFullScr', 'CleRec', 'AmbFullScr', 'AmbRec', 'Promoters', 'Detractors', 'NPSfilled', 'PromPerc',
           'DetrPerc', 'SITEEMAIL', 'TOTAL_SALESID']]

fdf.head(10)

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,FeedbackFilled,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,...,CleRec,AmbFullScr,AmbRec,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,SITEEMAIL,TOTAL_SALESID
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,258.0,4552577.0,3464518.04,76.10,53.97,...,2326.0,2290.0,2183.0,214.0,9.0,237.0,90.30,3.80,asansolmgr@surakshanet.com,2366.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,256.0,4601248.0,3913940.77,85.06,31.51,...,2150.0,2200.0,2007.0,201.0,15.0,226.0,88.94,6.64,asansolmgr@surakshanet.com,2815.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,34.0,4303131.0,3391092.60,78.81,45.45,...,229.0,220.0,195.0,21.0,5.0,28.0,75.00,17.86,asansolmgr@surakshanet.com,2584.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,349.0,3179057.0,2912362.01,91.61,49.90,...,3358.0,3360.0,3324.0,328.0,9.0,340.0,96.47,2.65,bangaonmgr@surakshanet.com,1592.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,411.0,3213045.0,3539110.72,110.15,56.77,...,3940.0,3950.0,3901.0,393.0,7.0,403.0,97.52,1.74,bangaonmgr@surakshanet.com,1988.0
5,BAN,BANGAON,Barasat Zone,3495875.67,Apr-25,32.0,3004870.0,3495875.67,116.34,26.34,...,255.0,230.0,190.0,16.0,2.0,20.0,80.00,10.00,bangaonmgr@surakshanet.com,1890.0
6,BAR,BARASAT,Barasat Zone,12608480.70,Apr-25,347.0,12448605.0,12608480.70,101.28,42.66,...,3171.0,3200.0,3083.0,295.0,11.0,317.0,93.06,3.47,barasatmgr@surakshanet.com,5303.0
7,BAR,BARASAT,Barasat Zone,10888172.57,Jun-25,814.0,13170230.0,10888172.57,82.67,64.97,...,7588.0,7590.0,7336.0,718.0,29.0,772.0,93.01,3.76,barasatmgr@surakshanet.com,4706.0
8,BAR,BARASAT,Barasat Zone,13293663.51,May-25,1022.0,13311033.0,13293663.51,99.87,54.94,...,9467.0,9640.0,9240.0,896.0,42.0,975.0,91.90,4.31,barasatmgr@surakshanet.com,5887.0
9,BAS,BASHIRHAT,Barasat Zone,4153041.53,May-25,348.0,4416430.0,4153041.53,94.04,26.56,...,3306.0,3300.0,3256.0,324.0,8.0,336.0,96.43,2.38,basirhatmgr@surakshanet.com,2130.0


# **Page 2 - Centre Target**

In [28]:
def calculate_rating(row):
    if row['TargetProj'] >= 100.0 and row['TARGET'] > 0:
        return 5
    elif row['TargetProj'] > 98.0 and row['TARGET'] > 0:
        return 4
    elif row['TargetProj'] > 95.0 and row['TARGET'] > 0:
        return 3
    else:
        return 0
        
fdf['rating'] = fdf.apply(calculate_rating, axis= 1)
fdf.head(7)

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,FeedbackFilled,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,...,AmbRec,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,SITEEMAIL,TOTAL_SALESID,rating,CM_Incentive
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,258.0,4552577.0,3464518.04,76.10,53.97,...,2183.0,214.0,9.0,237.0,90.30,3.80,asansolmgr@surakshanet.com,2366.0,0,0.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,256.0,4601248.0,3913940.77,85.06,31.51,...,2007.0,201.0,15.0,226.0,88.94,6.64,asansolmgr@surakshanet.com,2815.0,0,0.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,34.0,4303131.0,3391092.60,78.81,45.45,...,195.0,21.0,5.0,28.0,75.00,17.86,asansolmgr@surakshanet.com,2584.0,0,0.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,349.0,3179057.0,2912362.01,91.61,49.90,...,3324.0,328.0,9.0,340.0,96.47,2.65,bangaonmgr@surakshanet.com,1592.0,0,0.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,411.0,3213045.0,3539110.72,110.15,56.77,...,3901.0,393.0,7.0,403.0,97.52,1.74,bangaonmgr@surakshanet.com,1988.0,5,12000.0
5,BAN,BANGAON,Barasat Zone,3495875.67,Apr-25,32.0,3004870.0,3495875.67,116.34,26.34,...,190.0,16.0,2.0,20.0,80.00,10.00,bangaonmgr@surakshanet.com,1890.0,5,10000.0
6,BAR,BARASAT,Barasat Zone,12608480.70,Apr-25,347.0,12448605.0,12608480.70,101.28,42.66,...,3083.0,295.0,11.0,317.0,93.06,3.47,barasatmgr@surakshanet.com,5303.0,5,10000.0


# **Page 3 - Feedback Rating**

In [24]:
def calculate_cm_incentive(row):
    sales_proj = row['SalesProj']
    center_target = row['TargetProj']
    
    if center_target < 95:
        return 0
    
    if center_target >= 110:
        if sales_proj < 1500000:
            return 3000 * 2
        elif sales_proj < 3500000:
            return 5000 * 2
        elif sales_proj < 5500000:
            return 6000 * 2
        elif sales_proj < 8500000:
            return 8000 * 2
        else:
            return 10000 * 2

    elif center_target >= 105:
        if sales_proj < 1500000:
            return 3000 * 1.5
        elif sales_proj < 3500000:
            return 5000 * 1.5
        elif sales_proj < 5500000:
            return 6000 * 1.5
        elif sales_proj < 8500000:
            return 8000 * 1.5
        else:
            return 10000 * 1.5

    elif center_target >= 100:
        if sales_proj < 1500000:
            return 3000
        elif sales_proj < 3500000:
            return 5000
        elif sales_proj < 5500000:
            return 6000
        elif sales_proj < 8500000:
            return 8000
        else:
            return 10000

    elif center_target >= 95:
        if sales_proj < 1500000:
            return 3000 / 4
        elif sales_proj < 3500000:
            return 5000 / 4
        elif sales_proj < 5500000:
            return 6000 / 4
        elif sales_proj < 8500000:
            return 8000 / 4
        else:
            return 10000 / 4

    return None  


fdf['CM_Incentive'] = fdf.apply(calculate_cm_incentive, axis=1)

In [27]:
fdf.head(7)

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,FeedbackFilled,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,...,AmbRec,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,SITEEMAIL,TOTAL_SALESID,rating,CM_Incentive
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,258.0,4552577.0,3464518.04,76.10,53.97,...,2183.0,214.0,9.0,237.0,90.30,3.80,asansolmgr@surakshanet.com,2366.0,0,0.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,256.0,4601248.0,3913940.77,85.06,31.51,...,2007.0,201.0,15.0,226.0,88.94,6.64,asansolmgr@surakshanet.com,2815.0,0,0.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,34.0,4303131.0,3391092.60,78.81,45.45,...,195.0,21.0,5.0,28.0,75.00,17.86,asansolmgr@surakshanet.com,2584.0,0,0.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,349.0,3179057.0,2912362.01,91.61,49.90,...,3324.0,328.0,9.0,340.0,96.47,2.65,bangaonmgr@surakshanet.com,1592.0,0,0.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,411.0,3213045.0,3539110.72,110.15,56.77,...,3901.0,393.0,7.0,403.0,97.52,1.74,bangaonmgr@surakshanet.com,1988.0,5,12000.0
5,BAN,BANGAON,Barasat Zone,3495875.67,Apr-25,32.0,3004870.0,3495875.67,116.34,26.34,...,190.0,16.0,2.0,20.0,80.00,10.00,bangaonmgr@surakshanet.com,1890.0,5,10000.0
6,BAR,BARASAT,Barasat Zone,12608480.70,Apr-25,347.0,12448605.0,12608480.70,101.28,42.66,...,3083.0,295.0,11.0,317.0,93.06,3.47,barasatmgr@surakshanet.com,5303.0,5,10000.0


In [26]:
fdf.to_excel(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\cmp.xlsx", index= False)

In [30]:
def final_cm_incentive(row):
    if row['SRConvperc'] > 50 and row['NPSScore'] > 85 and row['CAScore'] > 90 and row['feedback_perticipation'] > 10:
        return row['CM_Incentive']
    else:
        return 0

fdf['final_incentive'] = fdf.apply(final_cm_incentive, axis= 1)

In [31]:
fdf

,SITEID,COSTCENTER_x,SUBZONENAME,SalesProj,FM,FeedbackFilled,TARGET,ACTUAL_SALES,TargetProj,SRConvperc,...,Promoters,Detractors,NPSfilled,PromPerc,DetrPerc,SITEEMAIL,TOTAL_SALESID,rating,CM_Incentive,final_incentive
0,ASN,ASANSOL,Burdwan,3464518.04,Jun-25,258.0,4552577.0,3464518.04,76.10,53.97,...,214.0,9.0,237.0,90.30,3.80,asansolmgr@surakshanet.com,2366.0,0,0.0,0.0
1,ASN,ASANSOL,Burdwan,3913940.77,May-25,256.0,4601248.0,3913940.77,85.06,31.51,...,201.0,15.0,226.0,88.94,6.64,asansolmgr@surakshanet.com,2815.0,0,0.0,0.0
2,ASN,ASANSOL,Burdwan,3391092.60,Apr-25,34.0,4303131.0,3391092.60,78.81,45.45,...,21.0,5.0,28.0,75.00,17.86,asansolmgr@surakshanet.com,2584.0,0,0.0,0.0
3,BAN,BANGAON,Barasat Zone,2912362.01,Jun-25,349.0,3179057.0,2912362.01,91.61,49.90,...,328.0,9.0,340.0,96.47,2.65,bangaonmgr@surakshanet.com,1592.0,0,0.0,0.0
4,BAN,BANGAON,Barasat Zone,3539110.72,May-25,411.0,3213045.0,3539110.72,110.15,56.77,...,393.0,7.0,403.0,97.52,1.74,bangaonmgr@surakshanet.com,1988.0,5,12000.0,12000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,TLG,TOLLYGUNGE,South Kol,4993913.18,Jun-25,404.0,6141119.0,4993913.18,81.32,46.89,...,360.0,11.0,383.0,93.99,2.87,tollygungemgr@surakshanet.com,3407.0,0,0.0,0.0
174,TLG,TOLLYGUNGE,South Kol,5470502.47,Apr-25,41.0,5804634.0,5470502.47,94.24,22.91,...,19.0,3.0,26.0,73.08,11.54,tollygungemgr@surakshanet.com,3663.0,0,0.0,0.0
175,TML,TAMLUK LAB,Midnapore,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0
176,ULB,Uluberia,Howrah/Hooghly,23840.00,None,NaN,NaN,23840.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0
